### LIBRERÍAS

In [2]:
# Procesamiento
import numpy as np
import pandas as pd
import ast  # Para oasar disccionarios en formmato string a formato diccionario
import os

# API y web scraping
from time import sleep

import requests
from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.common.by import By

# Visualizaciones
import plotly.express as px
import folium
from geopy.distance import geodesic
from geopy.geocoders import Nominatim

# Redes neuronales
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, SimpleRNN, Dropout, LSTM

### WEB SCRAPING IDAE

Fuente: https://coches.idae.es/base-datos

In [144]:
def inicio_scraping(endpoint):
    ### Driver
    chrome_driver = "chromedriver.exe"

    ### SE ABRE ENLACE
    browser = webdriver.Chrome(chrome_driver)

    browser.get(endpoint)

    browser.maximize_window()
    sleep(1)

    ### SE LE DA AL BOTON DE BUSCAR
    browser.find_element(by = By.CSS_SELECTOR, value = "body > div.container-fluid > div.row > div.col-lg-9.col-md-12 > form > div.col.col-12.text-center > button").click()
    sleep(1)

    return browser

**Vehículos elegibles Programa MOVES III**

In [145]:
endpoint = "https://coches.idae.es/base-datos/vehiculos-elegibles-programa-MOVES-III"

In [ ]:
browser = inicio_scraping(endpoint)

# REALIZAR SCROLL HASTA EL FINAL DE LA PÁGINA
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
sleep(1)

### SE LE DA AL BOTON DE EXTRAER EN CSV/EXCELL
browser.find_element(by = By.CSS_SELECTOR, value = "#nedc > p > button.btn.btn-primary.btnExportarCSV.mr-3").click()
sleep(5)

# CERRAR EL NAVEGADOR
browser.quit()

**WPLT**

In [4]:
endpoint = "https://coches.idae.es/base-datos/marca-y-modelo"

In [ ]:
browser = inicio_scraping(endpoint)

### SE CARGA EL DESPLEGABLE
browser.find_element(by = By.CSS_SELECTOR, value = "#datos_wltp_length > label > select").click()
sleep(1)

### SELECCIONAR 100
browser.find_element(by = By.CSS_SELECTOR, value = "#datos_wltp_length > label > select > option:nth-child(4)").click()
sleep(1)

### DESCARGAR Y PASAR DE PAGINA
for pagina in range(1,169):
    # SCROLL ARRIBA DEL TODO
    browser.execute_script("window.scrollTo(0, 0);")
    sleep(1)
    # PULSAR DESCARGAR CSV
    browser.find_element(by = By.CSS_SELECTOR, value = "#datos_wltp_wrapper > div.dt-buttons > button.dt-button.buttons-csv.buttons-html5.btn.btn-primary.btn-sm.mb-2").click()
    sleep(1)
    # REALIZAR SCROLL HASTA EL FINAL DE LA PÁGINA
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(1)
    # AVANZAR DE PÁGINA
    browser.find_element(by = By.CSS_SELECTOR, value = "#datos_wltp_next > a").click()
    sleep(2)

# Continúa con lo siguiente

**Consumo eléctrico**

In [ ]:
# SCROLL ARRIBA DEL TODO
browser.execute_script("window.scrollTo(0, 0);")

# SELECCION DE CONSUMO ELÉCTRICO
browser.find_element(by = By.CSS_SELECTOR, value = "#elec-tab").click()
sleep(1)

# SCROLL UN POCO HACIA ABAJO
browser.execute_script("window.scrollTo(0, window.scrollY + 100);")  # Ajusta el valor (100 en este caso) según sea necesario
sleep(1)

### SE CARGA EL DESPLEGABLE
browser.find_element(by = By.CSS_SELECTOR, value = "#datos_elec_length > label > select").click()
sleep(1)

### SELECCIONAR 100
browser.find_element(by = By.CSS_SELECTOR, value = "#datos_elec_length > label > select > option:nth-child(4)").click()
sleep(1)

### DESCARGAR Y PASAR DE PAGINA
for pagina in range(1,169):

    # SCROLL ARRIBA DEL TODO
    browser.execute_script("window.scrollTo(0, 0);")
    sleep(1)
    # PULSAR DESCARGAR CSV
    browser.find_element(by = By.CSS_SELECTOR, value = "#datos_elec_wrapper > div.dt-buttons > button.dt-button.buttons-csv.buttons-html5.btn.btn-primary.btn-sm.mb-2").click()
    sleep(1)
    # REALIZAR SCROLL HASTA EL FINAL DE LA PÁGINA
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(1)
    # AVANZAR DE PÁGINA
    browser.find_element(by = By.CSS_SELECTOR, value = "#datos_elec_next > a").click()
    sleep(2)

# CERRAR EL NAVEGADOR
browser.quit()

**Obteniendo Clasificación energética**

In [ ]:
browser = inicio_scraping(endpoint)
### SE CARGA EL DESPLEGABLE
browser.find_element(by = By.CSS_SELECTOR, value = "#datos_wltp_length > label > select").click()
sleep(1)
### SELECCIONAR 100
browser.find_element(by = By.CSS_SELECTOR, value = "#datos_wltp_length > label > select > option:nth-child(4)").click()
sleep(1)

titulo = []
modelo_coche = []
for pagina in range(1,170):
    browser.execute_script("window.scrollTo(0, 0);")
    sleep(1)
    for calificacion in range(1,101):
        try:
            img = browser.find_element(by = By.CSS_SELECTOR, value = f"#datos_wltp > tbody > tr:nth-child({calificacion}) > td.dt-center > img")
            title = img.get_attribute("title")
            titulo.append(title)
            modelo = browser.find_element(by = By.CSS_SELECTOR, value = f"#datos_wltp > tbody > tr:nth-child({calificacion}) > td.sorting_1 > a")
            modelo_coche_texto = modelo.text
            modelo_coche.append(modelo_coche_texto)
        except:
            print(f"la página no tiene {calificacion} elementos" )
    try:
        browser.find_element(by = By.CSS_SELECTOR, value = "#datos_wltp_next > a").click()
    except:
        print("fin")
        break    
    sleep(2)
browser.quit()

df_clasificación_wltp = pd.DataFrame({"Modelo":modelo_coche, "Clasificación":titulo})
df_clasificación_wltp.to_csv("data/df_clasificación_wltp.csv", index=False)

### API OPEN CHARGE MAP

Fuente: https://openchargemap.org/site

**Extracción de códigos postales de España**

Fuente: https://github.com/inigoflores/ds-codigos-postales-ine-es/blob/master/data/codigos_postales_municipios.csv

In [24]:
df = pd.read_csv('./Data/codigos_postales_municipios.csv', dtype={'codigo_postal': str})

In [25]:
df

,codigo_postal,municipio_id,municipio_nombre
0,00043,43110,"Pobla de Massaluca, La"
1,00085,49064,Fariza
2,00633,26005,Albelda de Iregua
3,01001,1059,Vitoria-Gasteiz
4,01002,1059,Vitoria-Gasteiz
...,...,...,...
14603,52002,52001,Melilla
14604,52003,52001,Melilla
14605,52004,52001,Melilla
14606,52005,52001,Melilla


**Repartición de peticiones**

In [28]:
ana = df["codigo_postal"][:3653].to_list()
maria = df["codigo_postal"][3653:7305].to_list()
cristian = df["codigo_postal"][7305:10957].to_list()
daniel = df["codigo_postal"][10957:14609].to_list()

In [31]:
diccionario = {}

### Bucle que controla el tiempo de ejecución a la vez que entra en el endpoint del código postal correspondiente
for codigo in tqdm(ana):
    # Construcción del endpoint
    url = "https://api.openchargemap.io/v3/poi/"
    parametros = {
                    "output": "json",
                    "countrycode": "ES",
                    "maxresults": 500,
                    "postalcode": codigo,
                    "key": ""  # Tu clave aquí
                }
    
    try:
        # Petición
        respuesta = requests.get(url, params=parametros)
        respuesta.raise_for_status()  # Lanza una excepción si la solicitud falla

        # Extrayendo toda la información
        datos = respuesta.json()
        diccionario[codigo] = datos

    except requests.exceptions.RequestException as e:
        print("Error al hacer la solicitud:", e)

# Creación de un df para guardar la información extraida y no perderla
df = pd.DataFrame(diccionario)

df.to_csv("./Data/ana.csv")

100%|██████████| 3653/3653 [1:28:10<00:00,  1.45s/it]   


In [32]:
df.shape

(500, 2750)

### PROCESAMIENTO DE DATOS

**Concatenación por filas de csv scrapeados**

In [2]:
def concatenar(ruta_carpeta):
    # Obtener la lista de archivos en la carpeta
    archivos_csv = [archivo for archivo in os.listdir(ruta_carpeta) if archivo.endswith('.csv')]

    # Leer y concatenar los archivos CSV
    datos_concatenados = pd.concat([pd.read_csv(os.path.join(ruta_carpeta, archivo)) for archivo in archivos_csv], ignore_index=True)

    return datos_concatenados

In [3]:
wplt = concatenar("../data/wplt/")

In [4]:
# Comprobando repetidos
wplt.Modelo.value_counts().to_dict()

{'Peugeot Nuevo 2008 2008 Allure BlueHDi 130 S&S EAT8': 5,
 'Peugeot 308 308 5p Allure BlueHDI 130 S&S 6 Vel. MAN': 4,
 'Scania Rígido Urbano B6x2 280cv PHEV Híbrido enchufable': 4,
 'Peugeot 308 308 5p Allure PureTech 130 S&S EAT8': 4,
 'Scania Rígido Urbano B6x2*4 280cv PHEV Híbrido enchufable': 4,
 'Peugeot 3008 3008 Allure PureTech 130 S&S 6 Vel. MAN.': 3,
 'Opel Vivaro (MY20.0) Express M Incrementada 1400 kg 2.0 Diesel MT6 120 CV': 3,
 'Peugeot 5008 5008 Allure BlueHDi 130 S&S 6 Vel. MAN.': 3,
 'TESLA Model X Performance': 3,
 'Peugeot 5008 5008 Active PureTech 130 S&S 6vel MAN': 3,
 'Peugeot 5008 5008 Active BlueHDi 130 S&S 6 Vel. MAN.': 3,
 'TESLA Model X Gran autonomía': 3,
 'Peugeot 3008 3008 GT Line BlueHDi 130 S&S 6vel MAN': 3,
 'Opel Vivaro (MY20.0) Express M Incrementada 1400 kg 2.0 Diesel MT6 150 CV': 3,
 'Peugeot 5008 5008 Allure PureTech 130 S&S EAT8': 3,
 'Toyota PROACE VERSO Shuttle largo Llanta 16p (MY20)': 3,
 'Peugeot 3008 3008 Allure PureTech 130 S&S EAT8': 3,
 'P

In [5]:
wplt.describe()

,Clasificación Energética,Consumo Mínimo,Consumo Máximo,Emisiones Mínimo,Emisiones Máximo
count,0.0,13899.000000,13897.000000,13845.000000,13845.000000
mean,NaN,6.533197,7.114305,146.196482,159.007403
std,NaN,7.733629,8.378181,47.727940,53.067782
min,NaN,0.100000,0.400000,1.300000,1.300000
25%,NaN,5.200000,5.600000,124.000000,133.000000
50%,NaN,5.800000,6.300000,140.000000,152.000000
75%,NaN,6.900000,7.700000,172.000000,185.000000
max,NaN,193.000000,213.000000,374.000000,394.000000


**Comprobando si los que tienen un Consumo Mínimo mayor a 100 son datos correctos**

In [6]:
consumo_100 = wplt[(wplt["Consumo Mínimo"] > 100)]["Modelo"].unique()

emisiones_10 = wplt[(wplt["Emisiones Mínimo"] < 10)]["Modelo"].unique()

for i, n in zip(consumo_100, emisiones_10):
    if n not in consumo_100:
        print(n)

Mitsubishi Outlander PHEV
Peugeot 3008 3008 Allure HYBRID 225 e-EAT8
Peugeot 3008 3008 GT HYBRID 225 e-EAT8
Peugeot 3008 3008 GT HYBRID 300 e-EAT8
Peugeot 3008 3008 GT Line HYBRID 225 e-EAT8
Peugeot 3008 3008 GT Line HYBRID 300 e-EAT8
SKODA NUEVO SUPERB COMBI MY21 L&K 1,4 TSI iV P-HEV Máx. 160 kW (218 CV) DSG 6 vel.
SKODA NUEVO SUPERB COMBI MY21 Sportline 1,4 TSI iV P-HEV Máx. 160 kW (218 CV) DSG 6 vel.
SKODA NUEVO SUPERB COMBI MY21 Style 1,4 TSI iV P-HEV Máx. 160 kW (218 CV) DSG 6 vel.
SKODA NUEVO SUPERB MY21 L&K 1,4 TSI iV P-HEV Máx. 160 kW (218 CV) DSG 6 vel.
SKODA NUEVO SUPERB MY21 Sportline 1,4 TSI iV P-HEV Máx. 160 kW (218 CV) DSG 6 vel.
SKODA NUEVO SUPERB MY21 Style 1,4 TSI iV P-HEV Máx. 160 kW (218 CV) DSG 6 vel.
SKODA OCTAVIA A8 MY21 1,4 TSI iV P-HEV150 kW (200 CV)DSG 6 vel.
SKODA SUPERB COMBI IV MY21 1,4 TSI iV P-HEV Máx. 160 kW (218 CV) DSG 6 vel.


In [7]:
indice = wplt[(wplt["Consumo Mínimo"] > 100)].index

**Intercambiando valores en sus columnas correspondientes**

In [8]:
for i in indice:
    emin = wplt.loc[i, "Consumo Mínimo"]
    emax = wplt.loc[i, "Consumo Máximo"]
    cmin = wplt.loc[i, "Emisiones Mínimo"]
    cmax = wplt.loc[i, "Emisiones Máximo"]

    wplt.loc[i, "Consumo Mínimo"] = cmin
    wplt.loc[i, "Consumo Máximo"] = cmax
    wplt.loc[i, "Emisiones Mínimo"] = emin
    wplt.loc[i, "Emisiones Máximo"] = emax

In [9]:
wplt[(wplt["Consumo Mínimo"] > 100)]

,Modelo,Clasificación Energética,Consumo Mínimo,Consumo Máximo,Emisiones Mínimo,Emisiones Máximo


In [10]:
wplt.duplicated().sum()

272

In [11]:
# Borrando duplicados
wplt = wplt.drop_duplicates()

In [12]:
wplt.reset_index(inplace=True, drop=True)

**Comprobando si los coches que tienen nulos en todas las columnas menos modelo son eléctricos**

In [13]:
wplt[wplt.isnull().sum(axis=1) == 5]

,Modelo,Clasificación Energética,Consumo Mínimo,Consumo Máximo,Emisiones Mínimo,Emisiones Máximo
0,ALKE ATX 340 EDH Litio 20kwh,NaN,NaN,NaN,NaN,NaN
1,ALKE ATX 340 EDH plomo acido,NaN,NaN,NaN,NaN,NaN
2,ALKE ATX 340 EH Litio 10 kwh,NaN,NaN,NaN,NaN,NaN
3,ALKE ATX 340 EH Litio 20kwh,NaN,NaN,NaN,NaN,NaN
4,ALKE ATX 340 EH plomo acido,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
16619,ALKE ATX 340 E plomo acido,NaN,NaN,NaN,NaN,NaN
16620,ALKE ATX 340 ED Litio 10 kwh,NaN,NaN,NaN,NaN,NaN
16621,ALKE ATX 340 ED Litio 20kwh,NaN,NaN,NaN,NaN,NaN
16622,ALKE ATX 340 ED plomo acido,NaN,NaN,NaN,NaN,NaN


In [14]:
son_electricos = wplt[wplt.isnull().sum(axis=1) == 5]["Modelo"].to_list()

In [15]:
electricos = concatenar("../data/electricos/")

In [16]:
electricos.Modelo.value_counts().to_dict()

{'Peugeot Nuevo 2008 2008 Allure BlueHDi 130 S&S EAT8': 5,
 'Peugeot 308 308 5p Allure PureTech 130 S&S EAT8': 4,
 'Peugeot 308 308 5p Allure BlueHDI 130 S&S 6 Vel. MAN': 4,
 'Toyota PROACE VERSO Family medio Llanta 17p (MY20)': 3,
 'Peugeot 5008 5008 Allure BlueHDi 130 S&S 6 Vel. MAN.': 3,
 'Peugeot Nuevo 2008 2008 GT PureTech 155 S&S EAT8': 3,
 'Peugeot 5008 5008 Active BlueHDi 130 S&S 6 Vel. MAN.': 3,
 'Opel Vivaro (MY20.0) Express M Incrementada 1400 kg 2.0 Diesel MT6 150 CV': 3,
 'Peugeot 308 308 5p Allure BlueHDI 130 S&S EAT8': 3,
 'Peugeot 5008 5008 Active PureTech 130 S&S 6vel MAN': 3,
 'Peugeot 5008 5008 Allure BlueHDi 130 S&S EAT8': 3,
 'TESLA Model X Gran autonomía': 3,
 'Peugeot 3008 3008 Allure BlueHDi 130 S&S EAT8': 3,
 'Peugeot 3008 3008 Allure BlueHDi 130 S&S 6vel MAN.': 3,
 'Peugeot 5008 5008 Allure PureTech 130 S&S EAT8': 3,
 'Peugeot 5008 5008 GT BlueHDi 180 S&S EAT8': 3,
 'Fiat eDucato Combi 35 L2H2': 3,
 'Peugeot 308 308 SW Allure BlueHDI 130 S&S 6 Vel. MAN': 3,
 '

In [17]:
electricos.duplicated().sum()

346

In [18]:
electricos = electricos.drop_duplicates()

In [19]:
si_son_electricos = electricos[electricos["Motorización"] == "Eléctricos puros"]["Modelo"].to_list()

no_electricos_puros = electricos[electricos["Motorización"] != "Eléctricos puros"]["Modelo"].to_list()

In [20]:
tambien_son_electricos = []
que_son = []
for i in son_electricos:
    if i in si_son_electricos:
        tambien_son_electricos.append(i)
    if i in no_electricos_puros:
        que_son.append(i)

In [21]:
len(tambien_son_electricos)

2671

In [22]:
len(son_electricos)

2845

In [23]:
len(son_electricos) - len(tambien_son_electricos)

174

In [24]:
len(que_son)

174

In [25]:
electricos[(electricos['Modelo'].isin(que_son)) & (electricos["Motorización"] != "Híbridos enchufables")]

,Modelo,Clasificación Energética,Motorización,Categoría,MTMA (Kg),Consumo Eléctrico (kWh/100 km),Potencia eléctrica (kW),Autonomía eléctrica (km),Capacidad de la batería (kWh)
71,Dacia Dokker 67P SCe 80kW (110CV) GLP N1 (semi...,NaN,Gases licuados del petróleo (GLP),N1,1958,NaN,NaN,NaN,NaN
3116,MAN HOC - Magnus E + A69 LE Gas Natural,NaN,Gas natural,M3,19500,NaN,NaN,NaN,NaN
3118,MAN LIONS City 12G 280 Gas Natural,NaN,Gas natural,M3,19500,NaN,NaN,NaN,NaN
3119,MAN LIONS City 12G 320 Gas Natural,NaN,Gas natural,M3,19500,NaN,NaN,NaN,NaN
3121,MAN LIONS City 18G Gas Natural,NaN,Gas natural,M3,30000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
16640,Volvo Trucks FM 62PT3HA,NaN,Gas natural,N3,32000,NaN,NaN,NaN,NaN
16642,Volvo Trucks FM 62TR3HA,NaN,Gas natural,N3,33000,NaN,NaN,NaN,NaN
16644,Volvo Trucks FM 62TT3HA,NaN,Gas natural,N3,33000,NaN,NaN,NaN,NaN
16646,Volvo Trucks FM 64R3HA,NaN,Gas natural,N3,36000,NaN,NaN,NaN,NaN


In [26]:
no_son_hibridos = electricos[(electricos['Modelo'].isin(que_son)) & (electricos["Motorización"] != "Híbridos enchufables")]["Modelo"].unique()

**Eliminando los que no son eléctricos**

In [27]:
wplt = wplt[~wplt['Modelo'].isin(no_son_hibridos)]
electricos = electricos[~electricos["Modelo"].isin(no_son_hibridos)]

wplt = wplt.sort_values("Modelo")
electricos = electricos.sort_values("Modelo")

wplt.reset_index(inplace=True, drop=True)
electricos.reset_index(inplace=True, drop=True)

In [28]:
electricos.head(1)

,Modelo,Clasificación Energética,Motorización,Categoría,MTMA (Kg),Consumo Eléctrico (kWh/100 km),Potencia eléctrica (kW),Autonomía eléctrica (km),Capacidad de la batería (kWh)
0,AIWAYS U5 MAS861-WVTA/2WB/FL4,NaN,Eléctricos puros,M1,2135,15.8,55.0,400.0,63.0


In [29]:
wplt.head(1)

,Modelo,Clasificación Energética,Consumo Mínimo,Consumo Máximo,Emisiones Mínimo,Emisiones Máximo
0,AIWAYS U5 MAS861-WVTA/2WB/FL4,NaN,NaN,NaN,NaN,NaN


In [30]:
electricos.shape

(16400, 9)

In [31]:
wplt.shape

(16474, 6)

**Merge finales para unir dfs**

In [32]:
# Calcular la media de la columna MTMA por modelo en df2
media_por_modelo = electricos.groupby('Modelo')['MTMA (Kg)'].mean().reset_index()
# Realizar el merge para agregar la columna Media_MTMA_Categoria a df
df = pd.merge(wplt, media_por_modelo, on='Modelo', how='left')

motorizacion_por_modelo = electricos[["Modelo", "Motorización"]]
df = pd.merge(df, motorizacion_por_modelo, on = "Modelo", how = 'left')

consumo_por_modelo = electricos[["Modelo", "Consumo Eléctrico (kWh/100 km)"]]
df = pd.merge(df, consumo_por_modelo, on = "Modelo", how = 'left')

columnas_por_modelo = electricos[["Modelo", "Potencia eléctrica (kW)", "Autonomía eléctrica (km)", "Capacidad de la batería (kWh)", "Categoría"]]
df = pd.merge(df, columnas_por_modelo, on = "Modelo", how = 'left')

In [33]:
df.duplicated().sum()

2881

In [34]:
df.drop_duplicates(inplace=True)

In [35]:
df.head()

,Modelo,Clasificación Energética,Consumo Mínimo,Consumo Máximo,Emisiones Mínimo,Emisiones Máximo,MTMA (Kg),Motorización,Consumo Eléctrico (kWh/100 km),Potencia eléctrica (kW),Autonomía eléctrica (km),Capacidad de la batería (kWh),Categoría
0,AIWAYS U5 MAS861-WVTA/2WB/FL4,NaN,NaN,NaN,NaN,NaN,2135.0,Eléctricos puros,15.8,55.0,400.0,63.0,M1
1,AIWAYS U6 PRIME,NaN,NaN,NaN,NaN,NaN,2290.0,Eléctricos puros,16.0,160.0,405.0,63.0,M1
2,ALKE ATX 310 E plomo acido,NaN,NaN,NaN,NaN,NaN,1510.0,Eléctricos puros,11.0,14.0,75.0,10.0,N1
3,ALKE ATX 320 E plomo acido,NaN,NaN,NaN,NaN,NaN,1510.0,Eléctricos puros,11.0,14.0,75.0,10.0,N1
4,ALKE ATX 330 E plomo acido,NaN,NaN,NaN,NaN,NaN,1510.0,Eléctricos puros,11.0,14.0,75.0,10.0,N1


**Separando conceptos**

In [36]:
# Crear una nueva columna llamada 'consumo_gas_natural' y llenarla con valores de 'consumo_minimo' solo cuando 'motorizacion' es 'Gas Natural'
df['Consumo Mínimo (kg/100km)'] = df.loc[df['Motorización'] == 'Gas natural', 'Consumo Mínimo']

df['Consumo Máximo (kg/100km)'] = df.loc[df['Motorización'] == 'Gas natural', 'Consumo Máximo']

In [37]:
df.head(1)

,Modelo,Clasificación Energética,Consumo Mínimo,Consumo Máximo,Emisiones Mínimo,Emisiones Máximo,MTMA (Kg),Motorización,Consumo Eléctrico (kWh/100 km),Potencia eléctrica (kW),Autonomía eléctrica (km),Capacidad de la batería (kWh),Categoría,Consumo Mínimo (kg/100km),Consumo Máximo (kg/100km)
0,AIWAYS U5 MAS861-WVTA/2WB/FL4,NaN,NaN,NaN,NaN,NaN,2135.0,Eléctricos puros,15.8,55.0,400.0,63.0,M1,NaN,NaN


In [38]:
df.rename(columns={"Consumo Mínimo" : "Consumo Mínimo (l/100km)"}, inplace=True)
df.rename(columns={"Consumo Máximo" : "Consumo Máximo (l/100km)"}, inplace=True)

**Leer un csv con un encoding raro**

In [39]:
# Leyendo el csv de solo coches electricos del apartado "vehiculos elegibles"
archivo_csv = '../data/Vehículos_elegibles_Programa_MOVES_III.csv'

# Intenta diferentes codificaciones
try:
    df_elegibles = pd.read_csv(archivo_csv, encoding='utf-8', delimiter=';')
except UnicodeDecodeError:
    # Si falla con utf-8, intenta con otra codificación
    df_elegibles = pd.read_csv(archivo_csv, encoding='latin-1', delimiter=';')

In [40]:
df_elegibles.shape

(3073, 9)

In [41]:
df_elegibles.head()

,Modelo,Motorización,Categoría,Nº de Plazas,MTMA,Potencia eléctrica (kW),Consumo eléctrico (kWh/100km),Capacidad de la batería (kWh),Autonomía eléctrica (km)
0,Abarth Abarth 500 BEV Nuevo Abarth 500e Scorpi...,Eléctricos puros,M1,4,1410,113.0,18.1,42.0,253
1,Abarth Abarth 500 BEV Nuevo Abarth Cabrio 500e...,Eléctricos puros,M1,4,1435,113.0,18.8,42.0,242
2,Addax MT15n 55 km/h Caja flatbed,Eléctricos puros,N1,2,1910,12.5,11.4,14.4,114
3,Addax MT15n 55 km/h Cargo,Eléctricos puros,N1,2,1910,12.5,11.4,14.4,114
4,Addax MT15n 55 km/h Chasis,Eléctricos puros,N1,2,1910,12.5,11.4,14.4,114


In [42]:
# Renombrar la columna 'MTMA' a 'MTMA (Kg)'
df_elegibles.rename(columns={'MTMA': 'MTMA (Kg)'}, inplace=True)

In [43]:
df.Motorización.unique()

array(['Eléctricos puros', 'Gasolina', 'Gasóleo', 'Híbridos enchufables',
       'Híbridos de gasolina', 'Gas natural', 'Híbridos de gasóleo',
       'Gases licuados del petróleo (GLP)', 'Pila de combustible',
       'Autonomía extendida', nan], dtype=object)

**Comprobando cuantos coches nuevos vamos a añadir y añadiendo las columnas que faltan a los modelos del df original**

In [44]:
lista = ['Eléctricos puros', 'Híbridos enchufables', 'Pila de combustible','Autonomía extendida']

In [45]:
df[df["Motorización"].isin(lista)].shape

(3633, 15)

In [46]:
df_elegibles.shape

(3073, 9)

In [47]:
modelos_endf = df[df["Motorización"].isin(lista)]["Modelo"].to_list()
modelos_enelegibles = df_elegibles["Modelo"].to_list()

In [48]:
modelos_repetidos = []
modelos_sin_repetir = []
for i in modelos_enelegibles:
    if i in modelos_endf:
        modelos_repetidos.append(i)
    else:
        modelos_sin_repetir.append(i)

In [49]:
len(modelos_repetidos)

2959

In [51]:
df_electricos_repetidos = df_elegibles[df_elegibles["Modelo"].isin(modelos_repetidos)]

In [52]:
plazas_por_modelo = df_electricos_repetidos[["Modelo", "Nº de Plazas"]]

df = pd.merge(df, plazas_por_modelo, on = "Modelo", how = 'left')

df_electricos_nuevos = df_elegibles[df_elegibles["Modelo"].isin(modelos_sin_repetir)]

df.drop("Clasificación Energética", axis=1, inplace=True)

df.rename(columns={"Consumo Eléctrico (kWh/100 km)":"Consumo Eléctrico (kWh/100km)"}, inplace=True)

orden_columnas = df.columns

In [53]:
df_electricos_nuevos.columns

Index(['Modelo', 'Motorización', 'Categoría', 'Nº de Plazas', 'MTMA (Kg)',
       'Potencia eléctrica (kW)', 'Consumo eléctrico (kWh/100km)',
       'Capacidad de la batería (kWh)', 'Autonomía eléctrica (km)'],
      dtype='object')

In [54]:
df_electricos_nuevos["Consumo Mínimo (l/100km)"] = np.nan
df_electricos_nuevos["Consumo Máximo (l/100km)"] = np.nan
df_electricos_nuevos["Emisiones Mínimo"] = np.nan
df_electricos_nuevos["Emisiones Máximo"] = np.nan
df_electricos_nuevos["Consumo Mínimo (kg/100km)"] = np.nan
df_electricos_nuevos["Consumo Máximo (kg/100km)"] = np.nan

df_electricos_nuevos.rename(columns={"Consumo eléctrico (kWh/100km)":"Consumo Eléctrico (kWh/100km)"}, inplace=True)

df_electricos_nuevos = df_electricos_nuevos[orden_columnas]

C:\Users\enaat\AppData\Local\Temp\ipykernel_16964\263729757.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_electricos_nuevos["Consumo Mínimo (l/100km)"] = np.nan
C:\Users\enaat\AppData\Local\Temp\ipykernel_16964\263729757.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_electricos_nuevos["Consumo Máximo (l/100km)"] = np.nan
C:\Users\enaat\AppData\Local\Temp\ipykernel_16964\263729757.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

**Concat final que unifica todo el escrapeo**

In [55]:
# Concatenar por filas
df_concatenado = pd.concat([df, df_electricos_nuevos], ignore_index=True, sort=False)

In [56]:
df.shape[0] + df_electricos_nuevos.shape[0]

16909

In [57]:
df_concatenado.shape[0]

16909

In [58]:
df_concatenado.duplicated().sum()

168

In [59]:
df_concatenado.drop_duplicates(inplace=True)

In [60]:
df_concatenado.to_csv("../data/df_coches_escrapeo.csv", index=False)

### LIMPIEZA DATOS

In [84]:
df = pd.read_csv("../data/df_coches_escrapeo.csv")

In [85]:
df

,Modelo,Consumo Mínimo (l/100km),Consumo Máximo (l/100km),Emisiones Mínimo,Emisiones Máximo,MTMA (Kg),Motorización,Consumo Eléctrico (kWh/100km),Potencia eléctrica (kW),Autonomía eléctrica (km),Capacidad de la batería (kWh),Categoría,Consumo Mínimo (kg/100km),Consumo Máximo (kg/100km),Nº de Plazas
0,AIWAYS U5 MAS861-WVTA/2WB/FL4,NaN,NaN,NaN,NaN,2135.0,Eléctricos puros,15.8,55.0,400.0,63.0,M1,NaN,NaN,5.0
1,AIWAYS U6 PRIME,NaN,NaN,NaN,NaN,2290.0,Eléctricos puros,16.0,160.0,405.0,63.0,M1,NaN,NaN,5.0
2,ALKE ATX 310 E plomo acido,NaN,NaN,NaN,NaN,1510.0,Eléctricos puros,11.0,14.0,75.0,10.0,N1,NaN,NaN,2.0
3,ALKE ATX 320 E plomo acido,NaN,NaN,NaN,NaN,1510.0,Eléctricos puros,11.0,14.0,75.0,10.0,N1,NaN,NaN,2.0
4,ALKE ATX 330 E plomo acido,NaN,NaN,NaN,NaN,1510.0,Eléctricos puros,11.0,14.0,75.0,10.0,N1,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16736,ZERO MOTORCYCLES FXE,NaN,NaN,NaN,NaN,454.0,Eléctricos puros,8.4,15.0,106.0,7.2,L3e,NaN,NaN,2.0
16737,ZERO MOTORCYCLES SR,NaN,NaN,NaN,NaN,454.0,Eléctricos puros,11.5,40.0,152.0,14.4,L3e,NaN,NaN,2.0
16738,ZERO MOTORCYCLES SR/F,NaN,NaN,NaN,NaN,454.0,Eléctricos puros,11.5,40.0,163.0,15.6,L3e,NaN,NaN,2.0
16739,ZERO MOTORCYCLES SR/S Premium,NaN,NaN,NaN,NaN,454.0,Eléctricos puros,11.5,40.0,163.0,15.6,L3e,NaN,NaN,2.0


**Analizando qué hacer con los modelos repetidos**

In [62]:
df_value_count = df.Modelo.value_counts()
df_modelos_repetidos = df.groupby(["Modelo"]).filter(lambda x: df_value_count[x.name] > 1)

In [63]:
df_modelos_repetidos.head(1)

,Modelo,Consumo Mínimo (l/100km),Consumo Máximo (l/100km),Emisiones Mínimo,Emisiones Máximo,MTMA (Kg),Motorización,Consumo Eléctrico (kWh/100km),Potencia eléctrica (kW),Autonomía eléctrica (km),Capacidad de la batería (kWh),Categoría,Consumo Mínimo (kg/100km),Consumo Máximo (kg/100km),Nº de Plazas
130,Audi A1 SPORTBACK 25 TFSI 70 kW (95 CV) 5 vel....,5.6,5.8,127.0,131.0,1630.0,Gasolina,NaN,NaN,NaN,NaN,M1,NaN,NaN,NaN


**Calculando las medias de las columnas numéricas**

In [64]:
df_modelos_repetidos.columns

Index(['Modelo', 'Consumo Mínimo (l/100km)', 'Consumo Máximo (l/100km)',
       'Emisiones Mínimo', 'Emisiones Máximo', 'MTMA (Kg)', 'Motorización',
       'Consumo Eléctrico (kWh/100km)', 'Potencia eléctrica (kW)',
       'Autonomía eléctrica (km)', 'Capacidad de la batería (kWh)',
       'Categoría', 'Consumo Mínimo (kg/100km)', 'Consumo Máximo (kg/100km)',
       'Nº de Plazas'],
      dtype='object')

In [65]:
medias_modelo = df.groupby(["Modelo", "Motorización"])[['Consumo Mínimo (l/100km)', 'Consumo Máximo (l/100km)',
       'Emisiones Mínimo', 'Emisiones Máximo', 'Consumo Eléctrico (kWh/100km)', 'Potencia eléctrica (kW)',
       'Autonomía eléctrica (km)', 'Capacidad de la batería (kWh)', 'Consumo Mínimo (kg/100km)', 'Consumo Máximo (kg/100km)',
       'Nº de Plazas', 'MTMA (Kg)']].transform("mean")

In [66]:
medias_modelo

,Consumo Mínimo (l/100km),Consumo Máximo (l/100km),Emisiones Mínimo,Emisiones Máximo,Consumo Eléctrico (kWh/100km),Potencia eléctrica (kW),Autonomía eléctrica (km),Capacidad de la batería (kWh),Consumo Mínimo (kg/100km),Consumo Máximo (kg/100km),Nº de Plazas,MTMA (Kg)
0,NaN,NaN,NaN,NaN,15.8,55.0,400.0,63.0,NaN,NaN,5.0,2135.0
1,NaN,NaN,NaN,NaN,16.0,160.0,405.0,63.0,NaN,NaN,5.0,2290.0
2,NaN,NaN,NaN,NaN,11.0,14.0,75.0,10.0,NaN,NaN,2.0,1510.0
3,NaN,NaN,NaN,NaN,11.0,14.0,75.0,10.0,NaN,NaN,2.0,1510.0
4,NaN,NaN,NaN,NaN,11.0,14.0,75.0,10.0,NaN,NaN,2.0,1510.0
...,...,...,...,...,...,...,...,...,...,...,...,...
16736,NaN,NaN,NaN,NaN,8.4,15.0,106.0,7.2,NaN,NaN,2.0,454.0
16737,NaN,NaN,NaN,NaN,11.5,40.0,152.0,14.4,NaN,NaN,2.0,454.0
16738,NaN,NaN,NaN,NaN,11.5,40.0,163.0,15.6,NaN,NaN,2.0,454.0
16739,NaN,NaN,NaN,NaN,11.5,40.0,163.0,15.6,NaN,NaN,2.0,454.0


In [67]:
df.update(medias_modelo)

In [68]:
df

,Modelo,Consumo Mínimo (l/100km),Consumo Máximo (l/100km),Emisiones Mínimo,Emisiones Máximo,MTMA (Kg),Motorización,Consumo Eléctrico (kWh/100km),Potencia eléctrica (kW),Autonomía eléctrica (km),Capacidad de la batería (kWh),Categoría,Consumo Mínimo (kg/100km),Consumo Máximo (kg/100km),Nº de Plazas
0,AIWAYS U5 MAS861-WVTA/2WB/FL4,NaN,NaN,NaN,NaN,2135.0,Eléctricos puros,15.8,55.0,400.0,63.0,M1,NaN,NaN,5.0
1,AIWAYS U6 PRIME,NaN,NaN,NaN,NaN,2290.0,Eléctricos puros,16.0,160.0,405.0,63.0,M1,NaN,NaN,5.0
2,ALKE ATX 310 E plomo acido,NaN,NaN,NaN,NaN,1510.0,Eléctricos puros,11.0,14.0,75.0,10.0,N1,NaN,NaN,2.0
3,ALKE ATX 320 E plomo acido,NaN,NaN,NaN,NaN,1510.0,Eléctricos puros,11.0,14.0,75.0,10.0,N1,NaN,NaN,2.0
4,ALKE ATX 330 E plomo acido,NaN,NaN,NaN,NaN,1510.0,Eléctricos puros,11.0,14.0,75.0,10.0,N1,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16736,ZERO MOTORCYCLES FXE,NaN,NaN,NaN,NaN,454.0,Eléctricos puros,8.4,15.0,106.0,7.2,L3e,NaN,NaN,2.0
16737,ZERO MOTORCYCLES SR,NaN,NaN,NaN,NaN,454.0,Eléctricos puros,11.5,40.0,152.0,14.4,L3e,NaN,NaN,2.0
16738,ZERO MOTORCYCLES SR/F,NaN,NaN,NaN,NaN,454.0,Eléctricos puros,11.5,40.0,163.0,15.6,L3e,NaN,NaN,2.0
16739,ZERO MOTORCYCLES SR/S Premium,NaN,NaN,NaN,NaN,454.0,Eléctricos puros,11.5,40.0,163.0,15.6,L3e,NaN,NaN,2.0


In [69]:
df.duplicated().sum()

410

In [70]:
df.drop_duplicates(inplace=True)

In [71]:
df.Modelo.value_counts().to_dict()

{'Peugeot 308 308 SW GT BlueHDi 130 S&S EAT8': 2,
 'Goupil G5 E Litio Goupil G5 E Litio PB + HP 500': 2,
 'Goupil G4 Litio Goupil G4 Litio FURGON': 2,
 'Goupil G4 Litio Goupil G4 Litio CAJA SATÉLITE': 2,
 'Goupil G5 E plomo Goupil G5 E plomo MIXTO PB + ASP,': 2,
 'Mercedes-Benz Clase E Estate (S214) 300 e': 2,
 'Goupil G5 E plomo Goupil G5 E plomo MIXTO PB + HP 200': 2,
 'Hyundai Canarias Tucson FL 2018 1.6 CRDi Híbrido 48V 85 kW 115 CV MT6 2WD - Trend Sky': 2,
 'Goupil G5 E plomo Goupil G5 E plomo PB': 2,
 'Mercedes-Benz Clase E Berlina (W213) 300 e 4MATIC': 2,
 'Mercedes-Benz Clase E Berlina (W213) 300 e': 2,
 'Mercedes-Benz Clase E Berlina (W213) 300 de 4MATIC': 2,
 'Mercedes-Benz Clase E Berlina (W213) 300 de': 2,
 'Goupil G5 E plomo Goupil G5 E plomo PB + HP 500': 2,
 'Toyota LAND CRUISER 180 5P Manual 5 plazas (MY19)': 2,
 'Peugeot Nuevo 2008 2008 Allure BlueHDi 130 S&S EAT8': 2,
 'Hyundai Canarias Tucson FL 2018 1.6 CRDi Híbrido 48V 85 kW 115 CV MT6 2WD - Style': 2,
 'Goupil G4 

In [72]:
df_value_count = df.Modelo.value_counts()
df_modelos_repetidos = df.groupby(["Modelo"]).filter(lambda x: df_value_count[x.name] > 1)

In [73]:
motorizacion = df_modelos_repetidos.groupby('Modelo')['Motorización'].nunique().to_dict()

In [74]:
motorizacion

{'Audi A5 SPORTBACK 40 TDI 150 kW (205CV) S tronic 7 vel. Advanced MY21': 2,
 'Audi A5 SPORTBACK 40 TDI 150 kW (205CV) S tronic 7 vel. S line/Black Line MY21': 2,
 'Audi A5 SPORTBACK 50 TDI quattro 210 kW (286 CV) tiptronic 8 vel. S line/Black Line MY21': 2,
 'Goupil G4 Litio Goupil G4 Litio CAJA SATÉLITE': 1,
 'Goupil G4 Litio Goupil G4 Litio FURGON': 1,
 'Goupil G4 Litio Goupil G4 Litio MIXTO FURGON': 1,
 'Goupil G4 Litio Goupil G4 Litio MIXTO PB + ASPIR,': 1,
 'Goupil G4 Litio Goupil G4 Litio MIXTO PB + GRUA': 1,
 'Goupil G4 Litio Goupil G4 Litio MIXTO PB + HP 200': 1,
 'Goupil G4 Litio Goupil G4 Litio PB': 1,
 'Goupil G4 Litio Goupil G4 Litio PB + HP 500': 1,
 'Goupil G4 Litio Goupil G4 Litio PF': 1,
 'Goupil G4 plomo Goupil G4 plomo CAJA SATÉLITE': 1,
 'Goupil G4 plomo Goupil G4 plomo FURGON': 1,
 'Goupil G4 plomo Goupil G4 plomo MIXTO FURGON': 1,
 'Goupil G4 plomo Goupil G4 plomo MIXTO PB + ASPIR,': 1,
 'Goupil G4 plomo Goupil G4 plomo MIXTO PB + GRUA': 1,
 'Goupil G4 plomo Goupi

In [75]:
df_estudio = df_modelos_repetidos.groupby("Modelo").filter(lambda x: motorizacion[x.name] > 1)

In [76]:
len(df_estudio.Modelo.unique())

12

In [77]:
df_estudio.head()

,Modelo,Consumo Mínimo (l/100km),Consumo Máximo (l/100km),Emisiones Mínimo,Emisiones Máximo,MTMA (Kg),Motorización,Consumo Eléctrico (kWh/100km),Potencia eléctrica (kW),Autonomía eléctrica (km),Capacidad de la batería (kWh),Categoría,Consumo Mínimo (kg/100km),Consumo Máximo (kg/100km),Nº de Plazas
530,Audi A5 SPORTBACK 40 TDI 150 kW (205CV) S tron...,5.20,5.25,137.0,137.0,2125.0,Gasóleo,NaN,1.3,NaN,NaN,M1,NaN,NaN,NaN
532,Audi A5 SPORTBACK 40 TDI 150 kW (205CV) S tron...,5.20,5.25,137.0,137.0,2125.0,Híbridos de gasóleo,NaN,1.3,NaN,NaN,M1,NaN,NaN,NaN
538,Audi A5 SPORTBACK 40 TDI 150 kW (205CV) S tron...,5.25,5.60,137.0,147.0,2125.0,Híbridos de gasóleo,NaN,1.3,NaN,NaN,M1,NaN,NaN,NaN
540,Audi A5 SPORTBACK 40 TDI 150 kW (205CV) S tron...,5.25,5.60,137.0,147.0,2125.0,Gasóleo,NaN,1.3,NaN,NaN,M1,NaN,NaN,NaN
577,Audi A5 SPORTBACK 50 TDI quattro 210 kW (286 C...,7.20,7.25,187.5,190.0,2342.5,Híbridos de gasóleo,NaN,8.4,NaN,NaN,M1,NaN,NaN,NaN


In [78]:
modelos_unificar = df_estudio[df_estudio["Motorización"] == "Híbridos de gasóleo"]["Modelo"].to_list()
indices_cambiar = df[df["Modelo"].isin(modelos_unificar)].index
for i in indices_cambiar:
    df.loc[i, "Motorización"] = "Híbridos de gasóleo"

In [79]:
df.duplicated().sum()

9

In [80]:
df.drop_duplicates(inplace=True)

In [81]:
df_value_count = df.Modelo.value_counts()
df_value_count

Modelo
Goupil G4 plomo Goupil G4 plomo MIXTO PF + HP 200                                          2
Goupil G4 Litio Goupil G4 Litio MIXTO PB + GRUA                                            2
Goupil G5 E Litio Goupil G5 E Litio PB                                                     2
Goupil G5 E Litio Goupil G5 E Litio MIXTO PB + HP 200                                      2
Goupil G5 E Litio Goupil G5 E Litio MIXTO PB + ASP,                                        2
                                                                                          ..
Land Rover Range Rover 5.0 V8 S/C 525CV 4WD LWB Auto - Vogue/Autobiography - 5 asientos    1
Land Rover Range Rover 5.0 V8 S/C 525CV 4WD SWB Auto - Vogue/Autobiography - 4 asientos    1
Land Rover Range Rover 5.0 V8 S/C 525CV 4WD SWB Auto - Vogue/Autobiography - 5 asientos    1
Land Rover Range Rover 5.0 V8 S/C 565CV 4WD LWB Auto - SVAutobiography - 4 asientos        1
ZERO MOTORCYCLES SR/S Standard                                 

In [82]:
df_modelos_repetidos = df.groupby(["Modelo"]).filter(lambda x: df_value_count[x.name] > 1)
df_modelos_repetidos.shape

(114, 15)

In [83]:
categoria = df_modelos_repetidos.groupby('Modelo')['Categoría'].nunique().to_dict()
df_estudio = df_modelos_repetidos.groupby("Modelo").filter(lambda x: categoria[x.name] > 1)

df_estudio.Modelo.to_dict()

{3301: 'Goupil G4 Litio Goupil G4 Litio CAJA SATÉLITE',
 3302: 'Goupil G4 Litio Goupil G4 Litio CAJA SATÉLITE',
 3303: 'Goupil G4 Litio Goupil G4 Litio FURGON',
 3304: 'Goupil G4 Litio Goupil G4 Litio FURGON',
 3305: 'Goupil G4 Litio Goupil G4 Litio MIXTO FURGON',
 3306: 'Goupil G4 Litio Goupil G4 Litio MIXTO FURGON',
 3307: 'Goupil G4 Litio Goupil G4 Litio MIXTO PB + ASPIR,',
 3308: 'Goupil G4 Litio Goupil G4 Litio MIXTO PB + ASPIR,',
 3309: 'Goupil G4 Litio Goupil G4 Litio MIXTO PB + GRUA',
 3310: 'Goupil G4 Litio Goupil G4 Litio MIXTO PB + GRUA',
 3311: 'Goupil G4 Litio Goupil G4 Litio MIXTO PB + HP 200',
 3312: 'Goupil G4 Litio Goupil G4 Litio MIXTO PB + HP 200',
 3313: 'Goupil G4 Litio Goupil G4 Litio PB',
 3314: 'Goupil G4 Litio Goupil G4 Litio PB',
 3315: 'Goupil G4 Litio Goupil G4 Litio PB + HP 500',
 3316: 'Goupil G4 Litio Goupil G4 Litio PB + HP 500',
 3317: 'Goupil G4 Litio Goupil G4 Litio PF',
 3318: 'Goupil G4 Litio Goupil G4 Litio PF',
 3391: 'Goupil G4 plomo Goupil G4 pl

In [84]:
dummies_categoria = pd.get_dummies(df["Categoría"])
dummies_categoria = dummies_categoria[dummies_categoria.columns].astype(int)
df_concat = pd.concat([df["Modelo"], dummies_categoria], axis=1)

df_concat

,Modelo,L3e,L5e,L6e,L7e,M1,M2,M3,N1,N2,N3
0,AIWAYS U5 MAS861-WVTA/2WB/FL4,0,0,0,0,1,0,0,0,0,0
1,AIWAYS U6 PRIME,0,0,0,0,1,0,0,0,0,0
2,ALKE ATX 310 E plomo acido,0,0,0,0,0,0,0,1,0,0
3,ALKE ATX 320 E plomo acido,0,0,0,0,0,0,0,1,0,0
4,ALKE ATX 330 E plomo acido,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
16736,ZERO MOTORCYCLES FXE,1,0,0,0,0,0,0,0,0,0
16737,ZERO MOTORCYCLES SR,1,0,0,0,0,0,0,0,0,0
16738,ZERO MOTORCYCLES SR/F,1,0,0,0,0,0,0,0,0,0
16739,ZERO MOTORCYCLES SR/S Premium,1,0,0,0,0,0,0,0,0,0


In [85]:
df_concat[df_concat["Modelo"] == "Goupil G4 plomo Goupil G4 plomo MIXTO PF + HP 200"]

,Modelo,L3e,L5e,L6e,L7e,M1,M2,M3,N1,N2,N3
3403,Goupil G4 plomo Goupil G4 plomo MIXTO PF + HP 200,0,0,0,1,0,0,0,0,0,0
3404,Goupil G4 plomo Goupil G4 plomo MIXTO PF + HP 200,0,0,0,0,0,0,0,1,0,0


In [86]:
df_concat.Modelo.value_counts()

Modelo
Goupil G4 plomo Goupil G4 plomo MIXTO PF + HP 200                                          2
Goupil G4 Litio Goupil G4 Litio MIXTO PB + GRUA                                            2
Goupil G5 E Litio Goupil G5 E Litio PB                                                     2
Goupil G5 E Litio Goupil G5 E Litio MIXTO PB + HP 200                                      2
Goupil G5 E Litio Goupil G5 E Litio MIXTO PB + ASP,                                        2
                                                                                          ..
Land Rover Range Rover 5.0 V8 S/C 525CV 4WD LWB Auto - Vogue/Autobiography - 5 asientos    1
Land Rover Range Rover 5.0 V8 S/C 525CV 4WD SWB Auto - Vogue/Autobiography - 4 asientos    1
Land Rover Range Rover 5.0 V8 S/C 525CV 4WD SWB Auto - Vogue/Autobiography - 5 asientos    1
Land Rover Range Rover 5.0 V8 S/C 565CV 4WD LWB Auto - SVAutobiography - 4 asientos        1
ZERO MOTORCYCLES SR/S Standard                                 

In [87]:
# Agrupar por 'Modelo' y aplicar la función max para unificar las filas
df_concat = df_concat.groupby('Modelo').max().reset_index()

In [88]:
df_concat[df_concat["Modelo"] == "Goupil G4 plomo Goupil G4 plomo MIXTO PF + HP 200"]

,Modelo,L3e,L5e,L6e,L7e,M1,M2,M3,N1,N2,N3
3374,Goupil G4 plomo Goupil G4 plomo MIXTO PF + HP 200,0,0,0,1,0,0,0,1,0,0


In [92]:
def crear_dic(columna):
    diccionario = {}
    for i in range(0, df_concat.shape[0]):
        modelo_actual = df_concat.loc[i, "Modelo"]
        valor_actual = df_concat.loc[i, columna]
        
        # Agregar al diccionario si el modelo no está presente o si hay una regla específica para manejar duplicados
        if modelo_actual not in diccionario:
            diccionario[modelo_actual] = valor_actual
        else:
            # Puedes manejar duplicados de alguna manera específica, por ejemplo, concatenando los valores
            diccionario[modelo_actual] += f", {valor_actual}"
    
    return diccionario

In [93]:
L3e = crear_dic("L3e")
L5e = crear_dic("L5e")
L6e = crear_dic("L6e")
L7e = crear_dic("L7e")
M1 = crear_dic("M1")
M2 = crear_dic("M2")
M3 = crear_dic("M3")
N1 = crear_dic("N1")
N2 = crear_dic("N2")
N3 = crear_dic("N3")

In [94]:
L3e

{'AIWAYS U5 MAS861-WVTA/2WB/FL4': 0,
 'AIWAYS U6 PRIME': 0,
 'ALKE ATX 310 E plomo acido': 0,
 'ALKE ATX 320 E plomo acido': 0,
 'ALKE ATX 330 E plomo acido': 0,
 'ALKE ATX 330 ED plomo acido': 0,
 'ALKE ATX 330 EH plomo acido': 0,
 'ALKE ATX 330 EHD plomo acido': 0,
 'ALKE ATX 340 E Litio 10 kwh': 0,
 'ALKE ATX 340 E Litio 20kwh': 0,
 'ALKE ATX 340 E plomo acido': 0,
 'ALKE ATX 340 ED Litio 10 kwh': 0,
 'ALKE ATX 340 ED Litio 20kwh': 0,
 'ALKE ATX 340 ED plomo acido': 0,
 'ALKE ATX 340 EDH Litio 10 kwh': 0,
 'ALKE ATX 340 EDH Litio 20kwh': 0,
 'ALKE ATX 340 EDH plomo acido': 0,
 'ALKE ATX 340 EH Litio 10 kwh': 0,
 'ALKE ATX 340 EH Litio 20kwh': 0,
 'ALKE ATX 340 EH plomo acido': 0,
 'ALSTOM APTIS 2 Puertas dobles 350 kWh': 0,
 'ALSTOM APTIS 3 Puertas dobles 350 kWh': 0,
 'ARENA RES3 R30': 1,
 'ARENA VT3 R30': 1,
 'ARES R14 CAJA BASCULANTE': 0,
 'ARES R14 CAJA FIJA': 0,
 'ARES R14 CHASIS CABINA': 0,
 'ASKOLL NGS 3': 1,
 'ASKOLL NGS 3 ': 1,
 'ASKOLL PRO 70': 1,
 'ASKOLL PRO 70 ': 1,
 'A

In [95]:
df_concat.columns

Index(['Modelo', 'L3e', 'L5e', 'L6e', 'L7e', 'M1', 'M2', 'M3', 'N1', 'N2',
       'N3'],
      dtype='object')

In [96]:
for columna in df_concat.columns:
    if columna != "Modelo":
        df[columna] = np.nan

In [97]:
df.shape

(16322, 25)

In [98]:
df["L3e"] = df["Modelo"].map(L3e)
df["L5e"] = df["Modelo"].map(L5e)
df["L6e"] = df["Modelo"].map(L6e)
df["L7e"] = df["Modelo"].map(L7e)
df["M1"] = df["Modelo"].map(M1)
df["M2"] = df["Modelo"].map(M2)
df["M3"] = df["Modelo"].map(M3)
df["N1"] = df["Modelo"].map(N1)
df["N2"] = df["Modelo"].map(N2)
df["N3"] = df["Modelo"].map(N3)

In [109]:
df_wltp = pd.read_csv("../data/df_clasificación_wltp.csv")

In [110]:
# Eliminar el prefijo "Clasificación: " de la columna 'clasificacion'
df_wltp['Clasificación'] = df_wltp['Clasificación'].str.replace('Clasificación: ', '')

#Dummies
dummies_clasificacion = pd.get_dummies(df_wltp["Clasificación"])
dummies_clasificacion = dummies_clasificacion[dummies_clasificacion.columns].astype(int)
df_concat = pd.concat([df_wltp["Modelo"], dummies_clasificacion], axis=1)

#Agrupar por 'Modelo' y aplicar la función max para unificar las filas
df_concat = df_concat.groupby('Modelo').max().reset_index()

In [111]:
A = crear_dic("A")
B = crear_dic("B")
C = crear_dic("C")
D = crear_dic("D")
E = crear_dic("E")
F = crear_dic("F")
G = crear_dic("G")
sin_clasificacion = crear_dic("Sin clasificación")

df["A"] = np.nan
df["B"] =  np.nan
df["C"] =  np.nan
df["D"] =  np.nan
df["E"] =  np.nan
df["F"] =  np.nan
df["G"] =  np.nan
df["Sin clasificacion"] =  np.nan


df["A"] = df["Modelo"].map(A)
df["B"] = df["Modelo"].map(B)
df["C"] = df["Modelo"].map(C)
df["D"] = df["Modelo"].map(D)
df["E"] = df["Modelo"].map(E)
df["F"] = df["Modelo"].map(F)
df["G"] = df["Modelo"].map(G)
df["Sin clasificacion"] = df["Modelo"].map(sin_clasificacion)

In [338]:
df.shape

(16322, 25)

### ANÁLISIS DE DATOS

In [131]:
df = pd.read_csv("../data/df_coches_escrapeo.csv")

**¿Existe alguna correlación entre la potencia eléctrica y la autonomía en vehículos eléctricos?**

In [132]:
df = df[(df["MTMA (Kg)"] > 400) & (df["MTMA (Kg)"] < 3000)]
df = df[df["Motorización"].notnull()]

In [133]:
df = df[~(df["Motorización"] == "Pila de combustible")]
df = df[~(df["Motorización"] == "Autonomía extendida")]

In [134]:
df.loc[(df['Motorización'] == 'Gasolina') & (df['Consumo Mínimo (l/100km)'] == 60), 'Consumo Mínimo (l/100km)'] = 6

In [135]:
df.reset_index(inplace=True, drop=True)

In [76]:
df

,Modelo,Consumo Mínimo (l/100km),Consumo Máximo (l/100km),Emisiones Mínimo,Emisiones Máximo,MTMA (Kg),Motorización,Consumo Eléctrico (kWh/100km),Potencia eléctrica (kW),Autonomía eléctrica (km),...,N2,N3,A,B,C,D,E,F,G,Sin clasificacion
0,AIWAYS U5 MAS861-WVTA/2WB/FL4,NaN,NaN,NaN,NaN,2135.0,Eléctricos puros,15.8,55.0,400.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,AIWAYS U6 PRIME,NaN,NaN,NaN,NaN,2290.0,Eléctricos puros,16.0,160.0,405.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,ALKE ATX 310 E plomo acido,NaN,NaN,NaN,NaN,1510.0,Eléctricos puros,11.0,14.0,75.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,ALKE ATX 320 E plomo acido,NaN,NaN,NaN,NaN,1510.0,Eléctricos puros,11.0,14.0,75.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,ALKE ATX 330 E plomo acido,NaN,NaN,NaN,NaN,1510.0,Eléctricos puros,11.0,14.0,75.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13360,ZERO MOTORCYCLES FXE,NaN,NaN,NaN,NaN,454.0,Eléctricos puros,8.4,15.0,106.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13361,ZERO MOTORCYCLES SR,NaN,NaN,NaN,NaN,454.0,Eléctricos puros,11.5,40.0,152.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13362,ZERO MOTORCYCLES SR/F,NaN,NaN,NaN,NaN,454.0,Eléctricos puros,11.5,40.0,163.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13363,ZERO MOTORCYCLES SR/S Premium,NaN,NaN,NaN,NaN,454.0,Eléctricos puros,11.5,40.0,163.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
df.columns

Index(['Modelo', 'Consumo Mínimo (l/100km)', 'Consumo Máximo (l/100km)',
       'Emisiones Mínimo', 'Emisiones Máximo', 'MTMA (Kg)', 'Motorización',
       'Consumo Eléctrico (kWh/100km)', 'Potencia eléctrica (kW)',
       'Autonomía eléctrica (km)', 'Capacidad de la batería (kWh)',
       'Categoría', 'Consumo Mínimo (kg/100km)', 'Consumo Máximo (kg/100km)',
       'Nº de Plazas', 'L3e', 'L5e', 'L6e', 'L7e', 'M1', 'M2', 'M3', 'N1',
       'N2', 'N3', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'Sin clasificacion'],
      dtype='object')

In [78]:
def funcion_correlacion_potencia_autonomia_e():
    df_filtrado = df[['Potencia eléctrica (kW)', 'Autonomía eléctrica (km)']].dropna()
    correlation_matrix = df_filtrado.corr()
    fig = px.imshow(
                    correlation_matrix,
                    x=correlation_matrix.columns,
                    y=correlation_matrix.columns,
                    color_continuous_scale='Viridis',
                    labels=dict(color='Correlación'),
                    )

    # Agrega ajustes
    fig.update_layout(title='Mapa de Correlación', width=500, height=500, xaxis=dict(tickangle=90))

    # Muestra el mapa de correlación
    fig.show()

In [79]:
funcion_correlacion_potencia_autonomia_e()

In [80]:
def funcion_scater_potencia_autonomia_e(motorizacion):
    df_filtrado = df[['Potencia eléctrica (kW)', 'Autonomía eléctrica (km)', 'Motorización']].dropna()
    df_filtrado = df_filtrado[df_filtrado["Motorización"].isin(motorizacion)]
    fig = px.scatter(
                    df_filtrado,
                    x=df_filtrado[df_filtrado.columns[0]], 
                    y=df_filtrado[df_filtrado.columns[1]], 
                    color=df_filtrado.columns[2])

    # Añadir título
    fig.update_layout(title_text='Gráfico de Líneas con Dos Columnas')

    # Mostrar el gráfico interactivo
    fig.show()

In [111]:
lista_posibles_motorizaciones = ['Eléctricos puros','Híbridos enchufables', 'Híbridos de gasolina']

In [82]:
funcion_scater_potencia_autonomia_e(lista_posibles_motorizaciones)

Los vehículos eléctricos puros tienen la mayor autonomía eléctrica, seguida de los híbridos enchufables con autonomía extendida. Los híbridos de gasolina y las pilas de combustible tienen una autonomía eléctrica significativamente menor.

Es importante tener en cuenta que la autonomía eléctrica de un vehículo puede variar en función de factores como el estilo de conducción, las condiciones climáticas y el peso de la carga. Por lo tanto, es importante consultar la información del fabricante para obtener información más precisa sobre la autonomía eléctrica de un vehículo concreto. Además en un clima mediterráneo, con veranos calurosos e inviernos suaves, el aire acondicionado puede reducir la autonomía eléctrica de un vehículo. Por lo tanto, los vehículos eléctricos con una mayor autonomía eléctrica pueden ser una mejor opción para los conductores que viven en la Comunidad Valenciana.

Además, la Comunidad Valenciana cuenta con una red de estaciones de carga pública en expansión. Sin embargo, es importante planificar los viajes con antelación para asegurarse de que hay estaciones de carga disponibles a lo largo del trayecto.

**¿Cómo varía la potencia eléctrica entre diferentes modelos de vehículos eléctricos?**

In [140]:
def funcion_potencia_barras_e(motorizacion):
    # Crear un gráfico de barras con Plotly Express
    df_filtrado = df[(df["Motorización"] == motorizacion) & (df["Potencia eléctrica (kW)"].notna())]
    figura = px.histogram(df_filtrado, 
                          x='Potencia eléctrica (kW)', 
                          title=motorizacion,
                          nbins=100)

    # Mostrar el gráfico
    figura.show()

In [141]:
def funcion_potencia_caja_e(motorizacion):
    df_filtrado = df[(df["Motorización"] == motorizacion) & (df["Potencia eléctrica (kW)"].notna())]
    fig = px.box(df_filtrado, 
                 y = 'Potencia eléctrica (kW)', 
                 title = motorizacion)
    fig.show()

In [143]:
for motorizacion in lista_posibles_motorizaciones:
    funcion_potencia_barras_e(motorizacion)
    funcion_potencia_caja_e(motorizacion)
    print(input("Conclusion:"))

El histograma muestra que la mayoría de los coches eléctricos puros en España tienen una potencia eléctrica de entre 200 y 400 kW. Esto significa que la mayoría de los coches eléctricos puros tienen una potencia similar a la de un coche de gasolina o diésel de tamaño medio.  El histograma también muestra que hay una pequeña proporción de coches eléctricos puros con una potencia eléctrica superior a 400 kW. Estos coches suelen ser deportivos o de lujo, y tienen una potencia suficiente para acelerar rápidamente y alcanzar velocidades altas.  A partir de estos datos, se pueden extraer las siguientes conclusiones:  La mayoría de los coches eléctricos puros en España tienen una potencia eléctrica suficiente para satisfacer las necesidades de la mayoría de los conductores. Hay una pequeña proporción de coches eléctricos puros con una potencia eléctrica superior a la media, que están diseñados para conductores que buscan un rendimiento superior. En general, los datos del histograma muestran q

El histograma muestra que la mayoría de los coches híbridos enchufables en España tienen una potencia eléctrica de entre 100 y 200 kW. Esto significa que la mayoría de los coches híbridos enchufables tienen una potencia similar a la de un coche de gasolina o diésel de tamaño compacto.  El histograma también muestra que hay una pequeña proporción de coches híbridos enchufables con una potencia eléctrica superior a 200 kW. Estos coches suelen ser de gama alta, y tienen una potencia suficiente para acelerar rápidamente y alcanzar velocidades altas. En comparación con el histograma de los coches eléctricos puros, se puede observar que la mayoría de los coches híbridos enchufables tienen una potencia eléctrica inferior. Esto se debe a que los coches híbridos enchufables también tienen un motor de gasolina o diésel, que proporciona potencia adicional cuando es necesario.  También se puede observar que la distribución de la potencia eléctrica de los coches híbridos enchufables es más uniforme

El histograma muestra que la mayoría de los coches híbridos de gasolina en España tienen una potencia eléctrica de entre 40 y 80 kW. Esto significa que la mayoría de los coches híbridos de gasolina tienen una potencia similar a la de un coche de gasolina o diésel de tamaño compacto.  El histograma también muestra que hay una pequeña proporción de coches híbridos de gasolina con una potencia eléctrica superior a 80 kW. Estos coches suelen ser de gama alta, y tienen una potencia suficiente para acelerar rápidamente y alcanzar velocidades altas. En comparación con los histogramas de los coches eléctricos puros y los coches híbridos enchufables, se puede observar que la mayoría de los coches híbridos de gasolina tienen una potencia eléctrica inferior. Esto se debe a que los coches híbridos de gasolina solo tienen un motor eléctrico, que proporciona potencia adicional cuando es necesario.  También se puede observar que la distribución de la potencia eléctrica de los coches híbridos de gasol

    - Eléctricos puros
El histograma muestra que la mayoría de los coches eléctricos puros en España tienen una potencia eléctrica de entre 200 y 400 kW. Esto significa que la mayoría de los coches eléctricos puros tienen una potencia similar a la de un coche de gasolina o diésel de tamaño medio.  El histograma también muestra que hay una pequeña proporción de coches eléctricos puros con una potencia eléctrica superior a 400 kW. Estos coches suelen ser deportivos o de lujo, y tienen una potencia suficiente para acelerar rápidamente y alcanzar velocidades altas.  A partir de estos datos, se pueden extraer las siguientes conclusiones:  La mayoría de los coches eléctricos puros en España tienen una potencia eléctrica suficiente para satisfacer las necesidades de la mayoría de los conductores. Hay una pequeña proporción de coches eléctricos puros con una potencia eléctrica superior a la media, que están diseñados para conductores que buscan un rendimiento superior. En general, los datos del histograma muestran que los coches eléctricos puros están disponibles en una gama de potencias eléctricas para satisfacer las necesidades de diferentes tipos de conductores.

    - Hibridos enchufables
El histograma muestra que la mayoría de los coches híbridos enchufables en España tienen una potencia eléctrica de entre 100 y 200 kW. Esto significa que la mayoría de los coches híbridos enchufables tienen una potencia similar a la de un coche de gasolina o diésel de tamaño compacto.  El histograma también muestra que hay una pequeña proporción de coches híbridos enchufables con una potencia eléctrica superior a 200 kW. Estos coches suelen ser de gama alta, y tienen una potencia suficiente para acelerar rápidamente y alcanzar velocidades altas. En comparación con el histograma de los coches eléctricos puros, se puede observar que la mayoría de los coches híbridos enchufables tienen una potencia eléctrica inferior. Esto se debe a que los coches híbridos enchufables también tienen un motor de gasolina o diésel, que proporciona potencia adicional cuando es necesario.  También se puede observar que la distribución de la potencia eléctrica de los coches híbridos enchufables es más uniforme que la de los coches eléctricos puros. Esto se debe a que los fabricantes de coches híbridos enchufables ofrecen una gama más amplia de potencias eléctricas para satisfacer las necesidades de diferentes tipos de conductores.

    - Híbridos gasolina
El histograma muestra que la mayoría de los coches híbridos de gasolina en España tienen una potencia eléctrica de entre 40 y 80 kW. Esto significa que la mayoría de los coches híbridos de gasolina tienen una potencia similar a la de un coche de gasolina o diésel de tamaño compacto.  El histograma también muestra que hay una pequeña proporción de coches híbridos de gasolina con una potencia eléctrica superior a 80 kW. Estos coches suelen ser de gama alta, y tienen una potencia suficiente para acelerar rápidamente y alcanzar velocidades altas. En comparación con los histogramas de los coches eléctricos puros y los coches híbridos enchufables, se puede observar que la mayoría de los coches híbridos de gasolina tienen una potencia eléctrica inferior. Esto se debe a que los coches híbridos de gasolina solo tienen un motor eléctrico, que proporciona potencia adicional cuando es necesario.  También se puede observar que la distribución de la potencia eléctrica de los coches híbridos de gasolina es más uniforme que la de los coches eléctricos puros y los coches híbridos enchufables.

### ALGORITMO MAPA

In [2]:
df_cargadores = pd.read_csv("../data/df_cargadores.csv")

In [3]:
df_cargadores = df_cargadores[~((df_cargadores["latitude"] == 40.579369) & (df_cargadores["longitude"] == 17.036496))]
df_cargadores = df_cargadores[~((df_cargadores["latitude"] == 42.208634) & (df_cargadores["longitude"] == 24.043651))]
df_cargadores = df_cargadores[~((df_cargadores["latitude"] == 37.41559459787993) & (df_cargadores["longitude"] == -122.07737083821156))]

In [4]:
df_cargadores["cargadores"] = df_cargadores["cargadores"].apply(lambda x: ast.literal_eval(x.replace("nan", "False")))

In [5]:
df_cargadores.reset_index(drop=True, inplace=True)

In [6]:
df_cargadores["title"] = np.nan
df_cargadores["energia"] = np.nan
df_cargadores["formalName"] = np.nan
df_cargadores["isfastChargeCapable"] = np.nan

In [7]:
for punto, i in zip(df_cargadores['cargadores'], range(0, df_cargadores.shape[0] -1)):
    title = []
    formalname = []
    energia = []
    carga_rapida = []
    
    for cargador in punto:
        #print(i)

        title.append(cargador['title'])
        formalname.append(cargador['formalName'])
        energia.append(cargador['energia'])

        if cargador["isFastChargeCapable"] == True:
            carga_rapida.append("Sí")
        else:
            carga_rapida.append("No")
    
    df_cargadores.loc[i, "title"] = str(title)
    df_cargadores.loc[i, "energia"] = str(energia)
    df_cargadores.loc[i, "formalName"] = str(formalname)
    df_cargadores.loc[i, "isfastChargeCapable"] = str(carga_rapida)


In [8]:
df_cargadores = df_cargadores.fillna("Desconocido")

In [15]:
df_cargadores.to_csv("../data/df_cargadores.csv")

In [16]:
df_cargadores = pd.read_csv("../data/df_cargadores.csv")

In [217]:
codigo_postal = 28001
direccion = f"{str(codigo_postal)}, España"
radio = 100 # En kilometros

In [235]:
# Define el nombre de tu aplicación o proyecto
nombre_app = "Mi Aplicación de Geolocalización"

# Crea una instancia del geolocalizador y proporciona el nombre de tu aplicación como user_agent
geolocator = Nominatim(user_agent=nombre_app)

# Obtener las coordenadas de la dirección
location = geolocator.geocode(direccion)

# Imprimir las coordenadas
if location:
    localizacion_inicial = [location.latitude, location.longitude]

else:
    print("La dirección no se encontró.")

# Crear un mapa centrado en las coordenadas iniciales
mapa = folium.Map(location=localizacion_inicial, zoom_start=1000 / radio)

# Iterar sobre las filas del DataFrame y agregar marcadores al mapa
for index, row in df_cargadores.iterrows():
    coordenadas_punto = (row['latitude'], row['longitude'])
    distancia = geodesic(localizacion_inicial, coordenadas_punto).kilometers

    # Agregar marcador solo si la distancia es menor o igual a 1 km
    if distancia <= radio:
        
        html_popup = f'''
            <div style="min-width: 600px; max-width: 600px; font-size: 16px;">
                <p><strong>Dirección:</strong> {str(row["addressLine1"])}</p>
                <p><strong>Tipo de cargadores:</strong> {str(row["title"]).replace("[", "").replace("]", "").replace("'", "")}</p>
                <p><strong>Energía de cada cargador:</strong> {str(row["energia"]).replace("[", "").replace("]", "").replace("'", "")}</p>
                <p><strong>¿Tiene carga rápida?:</strong> {str(row["isfastChargeCapable"]).replace("[", "").replace("]", "").replace("'", "")}</p>
                <p><strong>Distancia a tu ubicación:</strong> {distancia:.2f} km</p>
            </div>
        '''
        folium.Marker(location=coordenadas_punto, popup=folium.Popup(html=html_popup)).add_to(mapa)


### SERIES TEMPORALES

In [13]:
df = pd.read_csv("../Data/Precios_Gasolina_y_Precios_Gasoleo.csv")

In [17]:
df.Año.unique()

array([2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023,
       2024], dtype=int64)

In [18]:
df

,Año,Semana,Precio Gasolina,Precio Gasoleo
0,2002,Semana 1,0.759,0.673
1,2002,Semana 2,0.759,0.673
2,2002,Semana 3,0.767,0.673
3,2002,Semana 4,0.762,0.667
4,2002,Semana 5,0.762,0.668
...,...,...,...,...
1135,2023,Semana 52,1.531,1.492
1136,2024,Semana 1,1.534,1.493
1137,2024,Semana 2,1.535,1.488
1138,2024,Semana 3,1.535,1.483


**Visualización serie temporal**

In [ ]:
# Line Plot
px.line(data_frame = df, x = "Fecha", y = "Energia Consumida Mw/h")

**Asegurarnos de que los años están ordenados y que las semanas por año también**

In [46]:
df = df.sort_values("Año")

In [55]:
# Eliminar la palabra "Semana" de la columna "Semana"
df["Semana"] = df["Semana"].replace("Semana ", "", regex=True)

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1140 entries, 0 to 1139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Año              1140 non-null   int64  
 1   Semana           1140 non-null   object 
 2   Precio Gasolina  1140 non-null   float64
 3   Precio Gasoleo   1140 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 35.8+ KB


In [58]:
df["Semana"] = df["Semana"].astype(int)

In [59]:
# Ordenar dentro de cada grupo (por año)
df = df.groupby('Año').apply(lambda x: x.sort_values('Semana')).reset_index(drop=True)

**Separar train y test sabiendo que los años están ordenados al igual que las semanas de cada año**

In [62]:
train = df["Precio Gasolina"][0:900].to_list()
test = df["Precio Gasolina"][901:1139].to_list()

In [73]:
def sacar_x_y(lista, T):

    T = T # Segmentos
    X = list()
    Y = list()

    for t in range(len(lista) - T):
        
        # Toma valores de X_test de t en t con stride de 1
        x = lista[t : t + T]
        X.append(x)
        
        # Toma los valores de t en t
        y = lista[t + T]
        Y.append(y)

    # Transformamos a np.array y ajustamos las dimensiones
    # Para entrar en el modelo debe de tener 3 dimensiones
    X = np.array(X).reshape(-1, T, 1)
    Y = np.array(Y)
    N = len(X)

    return X, Y, T

**Entrenando redes neuronales**

In [ ]:
# Modelo Simple 1

X_train, y_train, T = sacar_x_y(train, 10)
X_test, y_test, T = sacar_x_y(test, 10)

model = Sequential()

model.add(Input(shape = (T, 1)))

model.add(SimpleRNN(units = 512, activation = "relu"))


model.add(Dense(units = 256, activation = "relu"))
model.add(Dense(units = 128, activation = "relu")) 
model.add(Dense(units = 64, activation = "relu")) 
model.add(Dense(units = 32, activation = "relu")) 
model.add(Dense(units = 1))

model.compile(optimizer = "adam", loss = "mse")

#model.summary()

history = model.fit(x = X_train,
                    y = y_train,
                    validation_data = (X_test, y_test),
                    epochs = 200)

print("------------------------------------------------")

instancia = df["Precio Gasolina"][-10: -1].to_list()
ultima_semana = 1.58
instancia.append(ultima_semana)
instancia = np.array(instancia).reshape(-1, T, 1)

prediccion_semana_5 = model.predict(instancia)
prediccion_semana_5 = prediccion_semana_5[0][0]
prediccion_semana_5

In [ ]:
def perdida_conjunto_prueba(model):
    puntuacion = model.evaluate(X_test, y_test)
    return puntuacion

In [162]:
def funcion_perdida(history):
    # loss
    plt.plot(history.history["loss"], label = "loss")
    plt.plot(history.history["val_loss"], label = "val_loss")
    plt.legend()
    plt.ylim(0, 0.025)
    plt.show()


In [181]:
# Modelo Simple 2 MODELO GANADOR

modelo_2 = Sequential()

modelo_2.add(Input(shape = (T, 1)))

modelo_2.add(SimpleRNN(units = 512, activation = "relu"))


modelo_2.add(Dense(units = 256, activation = "relu"))
modelo_2.add(Dense(units = 128, activation = "relu")) 
modelo_2.add(Dense(units = 64, activation = "relu")) 
modelo_2.add(Dense(units = 32, activation = "relu")) 
modelo_2.add(Dense(units = 1))

modelo_2.compile(optimizer = "adam", loss = "mse")

#modelo_2.summary()

X, y, T = sacar_x_y(df["Precio Gasolina"].to_list(), 10)

history = modelo_2.fit(x = X,
                       y = y,
                       epochs = 200)

print("------------------------------------------")

instancia = df["Precio Gasolina"][-10: -1].to_list()
input_usuario = 1.58
instancia.append(input_usuario)
instancia = np.array(instancia).reshape(-1, T, 1)

prediccion_semana_5 = modelo_2.predict(instancia)
prediccion_semana_5 = prediccion_semana_5[0][0]
prediccion_semana_5

Epoch 1/200
36/36 [==============================] - 3s 17ms/step - loss: 0.1724
Epoch 2/200
36/36 [==============================] - 1s 18ms/step - loss: 0.0032
Epoch 3/200
36/36 [==============================] - 1s 16ms/step - loss: 0.0019
Epoch 4/200
36/36 [==============================] - 1s 16ms/step - loss: 0.0017
Epoch 5/200
36/36 [==============================] - 1s 16ms/step - loss: 0.0016
Epoch 6/200
36/36 [==============================] - 1s 18ms/step - loss: 0.0015
Epoch 7/200
36/36 [==============================] - 1s 15ms/step - loss: 0.0015
Epoch 8/200
36/36 [==============================] - 1s 15ms/step - loss: 0.0020
Epoch 9/200
36/36 [==============================] - 1s 15ms/step - loss: 0.0018
Epoch 10/200
36/36 [==============================] - 1s 16ms/step - loss: 0.0012
Epoch 11/200
36/36 [==============================] - 1s 17ms/step - loss: 0.0011
Epoch 12/200
36/36 [==============================] - 1s 16ms/step - loss: 0.0012
Epoch 13/200
36/36 [=====

1.591089

**Prueba de predicción de la última semana del dataframe**

In [191]:
instancia_3 = df["Precio Gasolina"][-12: -2].to_list()
ultima_semana_3 = df["Precio Gasolina"][-2:-1].values[0]
#instancia_3.append(ultima_semana_3)
instancia_3 = np.array(instancia_3).reshape(-1, T, 1)
print("------------------------------------------")
prediccion_semana_5 = modelo_2.predict(instancia_3)
prediccion_semana_5 = prediccion_semana_5[0][0]
print(ultima_semana_3, prediccion_semana_5)

------------------------------------------
1/1 [==============================] - 0s 30ms/step
1.535 1.5173123


In [178]:
# Modelo LSTM 1

modelo_3 = Sequential()

modelo_3.add(Input(shape = (T, 1)))

modelo_3.add(LSTM(units = 512, activation = "relu"))


modelo_3.add(Dense(units = 256, activation = "relu"))
modelo_3.add(Dense(units = 128, activation = "relu")) 
modelo_3.add(Dense(units = 64, activation = "relu")) 
modelo_3.add(Dense(units = 32, activation = "relu")) 
modelo_3.add(Dense(units = 1))

modelo_3.compile(optimizer = "adam", loss = "mse")

modelo_3.summary()

X_train_3, y_train_3, T = sacar_x_y(df["Precio Gasolina"].to_list(), 53)


history = modelo_3.fit(x = X_train_3,
                    y = y_train_3,
                    epochs = 10)

print("----------------------------------")

instancia_2 = df["Precio Gasolina"][-53: -1].to_list()
ultima_semana = 1.58
instancia_2.append(ultima_semana)
instancia_2 = np.array(instancia_2).reshape(-1, T, 1)

prediccion_semana_5 = modelo_3.predict(instancia_2)
prediccion_semana_5 = prediccion_semana_5[0][0]
prediccion_semana_5

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 512)               1052672   
                                                                 
 dense_25 (Dense)            (None, 256)               131328    
                                                                 
 dense_26 (Dense)            (None, 128)               32896     
                                                                 
 dense_27 (Dense)            (None, 64)                8256      
                                                                 
 dense_28 (Dense)            (None, 32)                2080      
                                                                 
 dense_29 (Dense)            (None, 1)                 33        
                                                                 
Total params: 1227265 (4.68 MB)
Trainable params: 1227

In [ ]:
# Modelo LSTM 2

X_train_4, y_train_4, T = sacar_x_y(train, 53)
X_test_4, y_test_4, T = sacar_x_y(test, 53)

model_4 = Sequential()

model_4.add(Input(shape = (T, 1)))

model_4.add(LSTM(units = 512, activation = "relu"))


model_4.add(Dense(units = 256, activation = "relu"))
model_4.add(Dense(units = 128, activation = "relu")) 
model_4.add(Dense(units = 64, activation = "relu")) 
model_4.add(Dense(units = 32, activation = "relu")) 
model_4.add(Dense(units = 1))

model_4.compile(optimizer = "adam", loss = "mse")

# model_4.summary()

history = model_4.fit(x = X_train_4,
                    y = y_train_4,
                    validation_data = (X_test_4, y_test_4),
                    epochs = 10)

print("----------------------------------")

instancia_2 = df["Precio Gasolina"][-53: -1].to_list()
ultima_semana = 1.58
instancia_2.append(ultima_semana)
instancia_2 = np.array(instancia_2).reshape(-1, T, 1)

prediccion_semana_5 = model_4.predict(instancia_2)
prediccion_semana_5 = prediccion_semana_5[0][0]
prediccion_semana_5

### ALGORITMO QUE COMPARA PRECIOS DE CONSUMO POR COCHE

In [3]:
df_coches = pd.read_csv("../data/df_coches_escrapeo.csv")

In [ ]:
# copia del df original a fin de realizar comparativas
df_coches = df.copy()

#Limpieza del df para quitar los camiones autobuses y otros vehiculos que no sean coches o furgonetas
df_coches = df_coches[(df_coches["MTMA (Kg)"] > 400) & (df_coches["MTMA (Kg)"] < 3000)] 
df_coches = df_coches[df_coches["Motorización"].notnull()]
df_coches = df_coches[~(df_coches["Motorización"] == "Pila de combustible")]
df_coches = df_coches[~(df_coches["Motorización"] == "Autonomía extendida")]
df_coches.reset_index(inplace = True , drop = True)
#Fin de la limpieza de los coches que no son turismos o furgonetas
 
#quitar los electricos puros que tienen consumo de combustion
df_coches.loc[df_coches['Motorización'] == 'Eléctricos puros', ['Consumo Mínimo (l/100km)', 'Consumo Máximo (l/100km)']] = 0